# **MECS6616 Spring 2024 - Project 5**

# **Introduction**

***IMPORTANT:***
- **Before starting, make sure to read the [Assignment Instructions](https://courseworks2.columbia.edu/courses/197115/pages/assignment-instructions) page on Courseworks to understand the workflow and submission requirements for this project.**

**FOR PROJECT 5!!!**
- Apart from the link to your notebook, you are also required to submit `q_network.pth` of Part 1 and `ppo_network.zip` (model checkpoints are loaded and saved by stable_baselines3 as zip files) of Part 2 to Coursework. You should put the link to your notebook in the comment entry

# Project Setup


In [ ]:
# DO NOT CHANGE

# There will be error messages from this command. You can ignore those error messages
# as long as you see "Successfully installed setuptools-65.5.0" at the end.

# After installing setuptools, a pop-up window will appear and you will be prompted
# to restart the notebook environment. Click on the restart environment button before continuing

!pip install setuptools==65.5.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 6.7 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
cvxpy 1.3.3 requires setuptools>65.5.1, but you have setuptools 65.5.0 which is incompatible.


**----------------------------**
**WAIT FOR NOTEBOOK TO RESTART**
**----------------------------**

In [1]:
# DO NOT CHANGE

# After running this cell, the folder 'mecs6616_sp23_project3' will show up in the file explorer on the left (click on the folder icon if it's not open)
# It may take a few seconds to appear
!git clone https://github.com/roamlab/mecs6616_sp24_project5.git

Cloning into 'mecs6616_sp24_project5'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 15 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (15/15), 9.22 KiB | 1.54 MiB/s, done.


In [2]:
# DO NOT CHANGE

# copy all needed files into the working directory. This is simply to make accessing files easier
!cp -av /content/mecs6616_sp24_project5/* /content/

'/content/mecs6616_sp24_project5/arm_dynamics_base.py' -> '/content/arm_dynamics_base.py'
'/content/mecs6616_sp24_project5/arm_dynamics.py' -> '/content/arm_dynamics.py'
'/content/mecs6616_sp24_project5/arm_env.py' -> '/content/arm_env.py'
'/content/mecs6616_sp24_project5/geometry.py' -> '/content/geometry.py'
'/content/mecs6616_sp24_project5/README.md' -> '/content/README.md'
'/content/mecs6616_sp24_project5/render.py' -> '/content/render.py'
'/content/mecs6616_sp24_project5/robot.py' -> '/content/robot.py'
'/content/mecs6616_sp24_project5/score.py' -> '/content/score.py'


In [3]:
# DO NOT CHANGE

# There will be error messages from this command. You can ignore those error messages
# as long as you see "Successfully installed gym-0.21.0 stable-baselines3-1.5.0" at the end.

!pip install wheel==0.38.4
!pip install gym==0.21.0 stable-baselines3==1.5.0

  Attempting uninstall: wheel
    Found existing installation: wheel 0.43.0
    Uninstalling wheel-0.43.0:
      Successfully uninstalled wheel-0.43.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.7/177.7 kB 10.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.

# Part 1: Implement DQN

For this part, you will implement DQN from scratch. You SHOULD NOT use any RL libraries.

## Starter Code Explanation
In addition to code you are already familiar with from the previous project (i.e. arm dynamics, etc.) we are providing an "Environment" in the `ArmEnv` class. The environment "wraps around" the arm dynamics and provides the key functions that an RL algorithm expects: reset(...) and step(...). The implementation of `ArmEnv` follows the [OpenAI Gym](https://www.gymlibrary.dev/api/core/) API standard. It is a standard that is accepeted by many RL libraries and allows for our problem to be easily solved with various RL libraries. Take a moment to familiarize yourself with these functions! See [here](https://www.gymlibrary.dev/api/core/) for more information on the definition of the reset(...) and step(...) functions.

Important notes:

* The ArmEnv expects an action similar to the one used previously: a vector with a torque for every arm joint. Thus, the native action space for this environment is high-dimensional, and continuous. DQN will require an action space that is 1-dimensional and discrete. You will need to convert between these. For example, you can have an action space of [0, 1, 2,] where each number just represents the identity of an action candidate, and a conversion dictionary {0: [-0.1, -0.1], 1: [0.1, 0.1], 2: [0, 0]}. Then, when the Q network output an action 1, it will be converted into [0.1, 0.1] and used by the environment. Note that this is just an example method to implement the conversion and you do not have to follow the same procedure.
* The observation provided by the environment will comprise the same state vector as before, to which we append the current position of the end_effector and the goal for the end-effector. Since your policy must learn to reach arbitrary goals, the goal must be provided as part of the observation. So the observation will consist of 8 values: 4 for the state, 2 for the pos_ee and 2 for the goal.
* The maximum episode length of the environment is 200 steps. Each step is simulated for 0.01 second. This should be used for both training and testing.
* The reward function of this environment is by default r(s, a) = - dist(pos_ee, goal)^2 where represents the negative square of L2 distance between the current position of the end-effector and the goal position.

### Arm Environment Example
You are encouraged to view the `arm_env.py` file to understand the `random_goal()`, `reset()` and `step()`  functions but do not modify the file.

The `env.reset()` method, will reset the arm in the vertically downwards position and set a new random goal by calling the `random_goal()` method. By understanding how the goals are set you could guide your training in that direction. You can also provide your own goal as a (2,1) array to the reset function as an argument. This could come handy later when training the model.

The `env.step()` function takes an action as a (2,1) shaped array and outputs the next observation, reward, done and info. `info` is a dictionary with pos_ee and vel_ee values. This can come handy if you attempt to do some reward engineering.

The cell below provides an example of random policy interacting with the ArmEnv for 50 steps (0.5 seconds)

In [4]:
from render import Renderer
from arm_env import ArmEnv
from robot import Robot
from arm_dynamics import ArmDynamics
import numpy as np

# DO NOT CHANGE arm parameters
arm = Robot(
        ArmDynamics(
            num_links=2,
            link_mass=0.1,
            link_length=1,
            joint_viscous_friction=0.1,
            dt=0.01,
	    			gravity=False
        )
    )
arm.reset()
# ------------------

env = ArmEnv(arm, gui=False)

# Passing our own defined goal to the reset function
# goal = np.array([[0.5], [-1.5]])
# obs = env.reset(goal)

# Resetting the environment without the goal will set a random goal position
obs = env.reset()

for _ in range(50):
  rand_action = np.random.uniform(-1.5, 1.5, (2,1))
  obs, reward, done, info = env.step(rand_action)

### QNetwork
This class defines the architecture of your network. You must fill in the __init__(...) function which defines your network, and the forward(...) function which performs the forward pass.

Your action space should be discrete, with whatever cardinality you decide. The size of the output layer of your Q-Network should thus be the same as the cardinality of your action space. When selecting an action, a policy must choose the one that has the highest estimated Q-value for the current state. As part of the QNetwork class, we are providing the function select_discrete_action(...) which does exactly that.

The arm environment itself however expects a 2-dimensional, continuous action vector. Therefore, when it comes to send an action to the environment, you must provide the kind of action the environment expects. It is your job to determine how to convert between the discrete action space of your Q-Network and the continuous action space of the arm. You do this by filling in the action_discrete_to_continuous(...) function in your QNetwork. You can expect to call the step function of the environment like this:

```
self.env.step(self.q_network.action_discrete_to_continuous(discrete_action))
```

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim



class QNetwork(nn.Module):
  def __init__(self, env):
    super(QNetwork, self).__init__()
    #--------- YOUR CODE HERE --------------
    self.fc1 = nn.Linear(env.observation_space.shape[0], 128)
    self.fc2 = nn.Linear(128, 128)
    self.fc3 = nn.Linear(128,20)



    #---------------------------------------

  def forward(self, x, device):
    #--------- YOUR CODE HERE --------------
    x = torch.FloatTensor(x)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x
    #---------------------------------------


  def select_discrete_action(self, obs, device):
    # Put the observation through the network to estimate q values for all possible discrete actions
    est_q_vals = self.forward(obs.reshape((1,) + obs.shape), device)
    # Choose the discrete action with the highest estimated q value
    discrete_action = torch.argmax(est_q_vals, dim=1).tolist()[0]
    return discrete_action

  def action_discrete_to_continuous(self, discrete_action):
    #--------- YOUR CODE HERE --------------
    #random assigned action is for somereason better
    act_dict={
        0:[0.25,-0.4],
        1:[-1.4,0.7],
        2:[-0.9,0.4],
        3:[-0.3,-0.15],
        4:[-0.6,1.3],
        5:[1,-0.6],
        6:[-0.6,-1.2],
        7:[-1.2,-1.4],
        8:[0.7,0.1],
        9:[1.3,-0.9],
        10:[0.1,0.25],
        11:[1.3,0.2],
        12:[-0.15,0],
        13:[1,-0.7],
        14:[0.2,1.2],
        15:[-0.3,0.7],
        16:[-0.8,-1.3],
        17:[0.4,0.2],
        18:[-0.5,1.2],
        19:[0.3,-0.2],
    }
    continuous_action = np.array(act_dict[discrete_action])
    return continuous_action
    #---------------------------------------


We provide you with code to use the replay buffer in your RL implementation. You do not need to change the ReplayBuffer class.
```
rb = ReplayBuffer()
```
After creating a ReplayBuffer object you can add samples in the buffer using `put()`:
```
rb.put((obs, action, reward, next_obs, done))
```
Take random samples from the buffer using:
```
obs, actions, rewards, next_obses, dones = rb.sample(batch_size)
```


In [6]:
import collections
import random
import numpy as np


class ReplayBuffer():
    def __init__(self, buffer_limit):
        self.buffer = collections.deque(maxlen=buffer_limit)

    def put(self, transition):
        self.buffer.append(transition)

    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []

        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append(a)
            r_lst.append(r)
            s_prime_lst.append(s_prime)
            done_mask_lst.append(done_mask)

        return np.array(s_lst), np.array(a_lst), \
               np.array(r_lst), np.array(s_prime_lst), \
               np.array(done_mask_lst)

### TrainDQN
Here, you must fill in the train(...) function that actually trains your network.

We are providing a helper function called save_model(...) that will save the current Q-network. Use this as you see fit.

To set one network equal to another one, you can use code like this:
```
target_network.load_state_dict(self.q_network.state_dict())
```

If you would like to be graded with a specific seed for the random number generators, make sure to change the default seed in the initialization of the TrainDQN class.

The time taken to train the model will depend mainly on how big is your model architecture and the number of episodes you run the training for. As a reference, the time taken to train a model on 1500 episodes, which passed all evaluation metrics was about an hour.
* Reference value for clipping the gradient value as mentioned in class: 0.2
* Reference value for a typical size of Replay Buffer: >10k
* Reference value for batch size while training: 64 - 512

Note that these are just reference values and larger is not always better as it may slow things down.

It is good practice in RL to ensure simpler things are working before complicating environments or training techniques.

If you think your training method is not working at all, you could pass a fixed goal to the `env.reset()` method during the training loop to ensure that your model is learning.

In [7]:
import time
from render import Renderer
from arm_env import ArmEnv
import numpy as np
from math import dist
import os
from geometry import polar2cartesian


class TrainDQN:

  def __init__(self, env, seed=4):
    torch.manual_seed(seed)
    np.random.seed(seed)
    self.env = env
    self.device = torch.device('cpu')
    self.q_network = QNetwork(env).to(self.device)
    self.target_network = QNetwork(env).to(self.device)
    self.target_network.load_state_dict(self.q_network.state_dict())
    self.rb=ReplayBuffer(10000)

  def save_model(self, episode_num, save_dir='models'):
    timestr = time.strftime("%Y-%m-%d_%H-%M-%S")
    model_dir = os.path.join(save_dir, timestr)
    if not os.path.exists(os.path.join(model_dir)):
      os.makedirs(os.path.join(model_dir))
    savepath = os.path.join(model_dir, f'q_network_ep_{episode_num:04d}.pth')
    torch.save(self.q_network.state_dict(), savepath)
    print(f'model saved to {savepath}\n')
    return savepath


  def train_one_step(self,BATCH_SIZE,GAMMA,max_grad,optimizer):

    obs_as_array, actions, rewards, next_obses_as_array, dones=self.rb.sample(BATCH_SIZE)
    #print(obs)
    #convert to tensors
    obs_as_tensor = torch.tensor(obs_as_array, dtype=torch.float32)
    act_as_tensor = torch.tensor(actions, dtype=torch.long)
    r_as_tensor = torch.tensor(rewards, dtype=torch.float32)
    next_obses_as_tensor = torch.tensor(next_obses_as_array, dtype=torch.float32)
    dones_as_tensor = torch.tensor(dones, dtype=torch.float32)

    #Compute Q values
    Q_A = self.q_network.forward(obs_as_tensor, self.device).gather(1, act_as_tensor.unsqueeze(1)).squeeze()
    Q_T_m = self.target_network.forward(next_obses_as_tensor, self.device).max(1)[0]
    Q_T_est = r_as_tensor + GAMMA * Q_T_m

    #Loss function
    loss = torch.mean(F.mse_loss(Q_A, Q_T_est))
    optimizer.zero_grad()
    loss.backward()
    nn.utils.clip_grad_norm_(self.q_network.parameters(), max_grad)
    optimizer.step()
    return loss.item()

  #test a specific model on a series of goals to get overall performance
  def test_model(self,model_path,goals):
    test_network = QNetwork(env).to(self.device)
    test_network.load_state_dict(torch.load(model_path))
    test_network.eval()
    total_return=0

    for goal in goals:
      done = False
      obs = env.reset(goal)

      episode_return = 0
      while not done:
        action = test_network.select_discrete_action(obs, self.device)
        action = test_network.action_discrete_to_continuous(action)
        new_obs, reward, done, info = env.step(action)
        episode_return += reward

        obs = new_obs
      #print('Episode return: ', episode_return)
      total_return+=episode_return
    total_return/=len(goals)

    return total_return

  #generate a set of random goals
  def generate_goals_series(self,len):
    radius_max = 2.0
    radius_min = 1.5
    angle_max = 0.5
    angle_min = -0.5
    goals=[]
    for i in range(len):
      radius = (radius_max - radius_min) * np.random.random() + radius_min
      angle = (angle_max - angle_min) * np.random.random() + angle_min
      angle -= np.pi / 2
      goal = polar2cartesian(radius, angle)
      goals.append(goal)

    return goals

  def train(self):

    #--------- YOUR CODE HERE --------------
    path_pool=[]

    num_epi=1500
    batch_size=128
    max_grad=3
    GAMMA=0.9
    eps_threshold=0.2
    eps_decay=0.95
    optimizer = torch.optim.Adam(self.q_network.parameters(), lr = 0.0005)
    reward_episode_max=float('-inf')

    #--------- TRAIN 1500 EPISODES --------------
    for episode in range(num_epi):
      obs=self.env.reset()
      reward_episode=0
      eps_threshold*=eps_decay
      while True:
        rand=np.random.uniform(low=0,high=1)
        if rand>eps_threshold:
          #with torch.no_grad():
          discrete_action = self.q_network.select_discrete_action(obs,self.device)
        else:
          discrete_action=np.random.randint(0,20)
        continuous_action = self.q_network.action_discrete_to_continuous(discrete_action)


        next_obs, next_r, next_done, _ = self.env.step(continuous_action)

        if next_done:
          break

        reward_episode+=next_r
        self.rb.put((obs, discrete_action, next_r, next_obs, next_done))

        if len(self.rb.buffer) >= 2000:
          self.train_one_step(batch_size,GAMMA,max_grad,optimizer)

        obs=next_obs

      print(f'End of episode {episode}, totel reward is: {reward_episode}')

      if episode%25==0:
        self.target_network.load_state_dict(self.q_network.state_dict())
      if reward_episode>-5 and episode>=800:
        save_path=self.save_model(episode_num=episode)
        path_pool.append(save_path)
    print('now all models have finished training')
    #--------- TRAINING FINISH, RETURN A SET OF GOOD MODEL --------------

    #--------- THEN CHOOSE BEST MODEL RUNNING TESTS --------------
    goals=self.generate_goals_series(20)
    max_reward=float('-inf')
    for path in path_pool:
      test_reward=self.test_model(path,goals)
      print(f'model tested: {path}\n')
      print(f'overall performance: {test_reward}\n\n')
      if test_reward>max_reward:
        max_reward=test_reward
        best_model_path=path


    return best_model_path







In [12]:
from robot import Robot
from arm_dynamics import ArmDynamics

# DO NOT CHANGE
# ---------------
arm = Robot(
        ArmDynamics(
            num_links=2,
            link_mass=0.1,
            link_length=1,
            joint_viscous_friction=0.1,
            dt=0.01,
	    			gravity=False
        )
    )
arm.reset()
env = ArmEnv(arm, gui=False)
tqdn = TrainDQN(env)
# ---------------

# Call your trin function here
model_path=tqdn.train()

Streaming output truncated to the last 5000 lines.
End of episode 218, totel reward is: -118.29092668266759
End of episode 219, totel reward is: -45.40520310440002
End of episode 220, totel reward is: -43.02989376855593
End of episode 221, totel reward is: -52.07646091235992
End of episode 222, totel reward is: -18.2242606310167
End of episode 223, totel reward is: -404.814359766083
End of episode 224, totel reward is: -58.793500932716896
End of episode 225, totel reward is: -57.515989672475804
End of episode 226, totel reward is: -197.88409579008342
End of episode 227, totel reward is: -80.45324112336198
End of episode 228, totel reward is: -74.31477627415723
End of episode 229, totel reward is: -264.8402320055236
End of episode 230, totel reward is: -32.112686764053706
End of episode 231, totel reward is: -40.67331400545908
End of episode 232, totel reward is: -79.7282909765223
End of episode 233, totel reward is: -16.288506999834397
End of episode 234, totel reward is: -11.548263129

To keep track of your experiments, it is good practice to plot and check how well is your model trained based on the returns vs episodes plot. With a large number of episodes, this  plot may look very jagged making it difficult to ascertain how well you are doing. We are proving code to smoothen out the plot by. This will take a large list of returns in every episode and plot a smoothened version of the list. Feel free to use it if it helps.
```
import seaborn as sns
returns = __
smoothing = 10

smoothened = [sum(returns[i:i+smoothing])/smoothing for i in range(0, len(returns), smoothing)]
sns.lineplot(smoothened)
```

### Load your model and test its performance
Change your model path and the goal to see how well your learnt model is performing

In [13]:
import collections
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
from render import Renderer
from arm_env import ArmEnv
import numpy as np
import os
from math import dist
import seaborn as sns
from robot import Robot
from arm_dynamics import ArmDynamics
from geometry import polar2cartesian


# DO NOT CHANGE arm parameters
arm = Robot(
        ArmDynamics(
            num_links=2,
            link_mass=0.1,
            link_length=1,
            joint_viscous_friction=0.1,
            dt=0.01,
	    			gravity=False
        )
    )
arm.reset()
# ------------------

env = ArmEnv(arm, gui=False)
model_path = model_path # Fill in the model_path
print(model_path)
device = torch.device('cpu')
qnet = QNetwork(env).to(device)
qnet.load_state_dict(torch.load(model_path))
qnet.eval()
goal = polar2cartesian(1.6, 0.25 - np.pi/2.0)
done = False
obs = env.reset(goal)
torch.save(qnet, ' q_network.pth')

episode_return = 0
while not done:
  action = qnet.select_discrete_action(obs, device)
  action = qnet.action_discrete_to_continuous(action)
  new_obs, reward, done, info = env.step(action)
  episode_return += reward

  pos_ee = info['pos_ee']
  vel_ee = info['vel_ee']
  dist = np.linalg.norm(pos_ee - goal)

  obs = new_obs
print('Episode return: ', episode_return)


models/2024-05-06_17-06-04/q_network_ep_1453.pth
Episode return:  -1.5164057455798456


### Grading and Evaluation
You will be evaluated on 5 different goal positions worth 1.5 points each. You must pass the best `model_path` for your network. The scoring function will run one episode for every goal position and find the total reward (aka return) for the episode. For every goal you get:

* 1 Point if `easy target < total reward < hard target`
* 1.5 Points if `hard target < total reward`

In [16]:
from score import compute_score
import torch.nn as nn
import torch
import torch.nn.functional as F
from render import Renderer
from arm_env import ArmEnv
from robot import Robot
from arm_dynamics import ArmDynamics
import numpy as np


# DO NOT CHANGE arm parameters
arm = Robot(
        ArmDynamics(
            num_links=2,
            link_mass=0.1,
            link_length=1,
            joint_viscous_friction=0.1,
            dt=0.01,
	    			gravity=False
        )
    )
arm.reset()
# ------------------

env = ArmEnv(arm, gui=False)
model_path = 'models/2024-05-06_17-06-04/q_network_ep_1453.pth'
device = torch.device('cpu')
qnet = QNetwork(env).to(device)
qnet.load_state_dict(torch.load(model_path))
qnet.eval()
score = compute_score(qnet, env, device)

---Computing score---

Goal 1:
Total reward: -1.2345310268992231
easy target: -7
hard target: -5
points: 1.5

Goal 2:
Total reward: -1.5164057455798456
easy target: -7
hard target: -5
points: 1.5

Goal 3:
Total reward: -2.547095029131439
easy target: -7
hard target: -5
points: 1.5

Goal 4:
Total reward: -9.733207651982383
easy target: -7
hard target: -5
points: 0

Goal 5:
Total reward: -4.8514713089341335
easy target: -10
hard target: -7
points: 1.5


Final score: 6.0


# Part 2: PPO with an open source RL library

In this part, you will use one of the most popular open source RL libraries ([Stable-Baselines3](https://stable-baselines3.readthedocs.io/en/master/)) to solve the same goal reaching problem as Part 1. We will use the same `ArmEnv` gym environment. The algorithm you should choose to use is PPO.

## PPO training

We provide the code to construct parallel environments. Parallel environments can be very useful if you have good CPUs and it can speed up training.

In [ ]:
# DO NOT CHANGE

from stable_baselines3.common.vec_env.subproc_vec_env import SubprocVecEnv
from stable_baselines3.common.vec_env.vec_monitor import VecMonitor
from copy import deepcopy
from robot import Robot
from arm_dynamics import ArmDynamics
from arm_env import ArmEnv

class EnvMaker:
    def __init__(self,  arm, seed):
        self.seed = seed
        self.arm = arm

    def __call__(self):
        arm = deepcopy(self.arm)
        env = ArmEnv(arm)
        env.seed(self.seed)
        return env

def make_vec_env(arm, nenv, seed):
    return VecMonitor(SubprocVecEnv([EnvMaker(arm, seed  + 100 * i) for i in range(nenv)]))

# conveniet function to create a robot arm
def make_arm():
    arm = Robot(
        ArmDynamics(
            num_links=2,
            link_mass=0.1,
            link_length=1,
            joint_viscous_friction=0.1,
            dt=0.01
        )
    )
    arm.reset()
    return arm


You will need to complete the code to train the policy using the [PPO class](https://stable-baselines3.readthedocs.io/en/master/modules/ppo.html) from stable_baselines3. We provide the code to generate the name of the directory to save the checkpoint, an example is `ppo_models/2024-04-13_01-14-13`. Your checkpoint model should be named `ppo_network.zip`. See the [save](https://stable-baselines3.readthedocs.io/en/master/modules/ppo.html#stable_baselines3.ppo.PPO.save) function. Training should take less than 40 minutes.

In [ ]:
from stable_baselines3.ppo import PPO
import os
import time
from stable_baselines3.common.utils import set_random_seed


# Default parameters
timesteps = 1000000
nenv = 8  # number of parallel environments. This can speed up training when you have good CPUs
seed = 8
batch_size = 2048

# Generate path of the directory to save the checkpoint
timestr = time.strftime("%Y-%m-%d_%H-%M-%S")
save_dir = os.path.join('ppo_network' )

# Set random seed
set_random_seed(seed)

# Create arm
arm = make_arm()

# Create parallel envs
vec_env = make_vec_env(arm=arm, nenv=nenv, seed=seed)

# ------ IMPLEMENT YOUR TRAINING CODE HERE ------------
model = PPO('MlpPolicy', vec_env, verbose = 1, batch_size=batch_size,learning_rate=0.0005)
model.learn(total_timesteps=timesteps)
model.save(f'{save_dir}')


#raise NotImplementedError

# Do not forget to save your model at the end of training

Using cpu device
-----------------------------------
| rollout/           |            |
|    ep_len_mean     | 200        |
|    ep_rew_mean     | -211.95427 |
| time/              |            |
|    fps             | 601        |
|    iterations      | 1          |
|    time_elapsed    | 27         |
|    total_timesteps | 16384      |
-----------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 200          |
|    ep_rew_mean          | -154.62141   |
| time/                   |              |
|    fps                  | 590          |
|    iterations           | 2            |
|    time_elapsed         | 55           |
|    total_timesteps      | 32768        |
| train/                  |              |
|    approx_kl            | 0.0040957723 |
|    clip_fraction        | 0.0306       |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.84        |
|    explained_varian

## Grading and evaluation

The total number of points for Part 2 is 7.5. We will evaluate your trained model on 5 random goal locations. For each test, we assign points based on the distance between the end effector and the goal location at the end of the episode.

- If 0 < distance < 0.05, you get 1.5 points.
- If 0.05 <= distance < 0.1, you get 1 point.
- If distance >= 0.1, you get 0 point.



In [ ]:
from score import score_policy
from stable_baselines3 import PPO
from stable_baselines3.common.utils import set_random_seed
from robot import Robot
from arm_dynamics import ArmDynamics
from render import Renderer
import time

# Set the path to your model
model_path = 'ppo_network.zip'

set_random_seed(seed=100)

# Create arm robot
arm = make_arm()

# Create environment
env = ArmEnv(arm, gui=False)
env.seed(100)

# Load and test policy
policy = PPO.load(model_path)
score_policy(policy, env)


--- Computing score ---

Goal 1: 1.5

Goal 2: 1.5

Goal 3: 1.5

Goal 4: 1.5

Goal 5: 1.5


---
Final score: 7.5/7.5
---


7.5